In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("mteb/tweet_sentiment_extraction")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3432 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset["train"])
df

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
...,...,...,...,...
26727,4eac33d1c0,wish we could come see u on Denver husband l...,0,negative
26728,4f4c4fc327,I`ve wondered about rake to. The client has ...,0,negative
26729,f67aae2310,Yay good for both of you. Enjoy the break - y...,2,positive
26730,ed167662a5,But it was worth it ****.,2,positive


#Tokenizer

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
  return tokenizer(examples["text"], padding= "max_length", truncation= True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/26732 [00:00<?, ? examples/s]

Map:   0%|          | 0/3432 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# Initialize our base model

In [ ]:
from transformers import GPT2ForSequenceClassification
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels = 3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Evaluate method

In [ ]:
# !pip install evaluate
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

def compute_metric(eval_pred):
  logits, labels = eval_pred # Destructing the prediction from the model
  predictions = np.argmax(logits, axis= -1)
  return metric.compute(predictions= predictions, references = labels)

#Fine tune using the trained method

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "test_trainer",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 4
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_test_dataset,
    compute_metrics = compute_metric
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: stevemichael681 (stevemichael681-jecrc-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.886000


TrainOutput(global_step=750, training_loss=0.7538310953776042, metrics={'train_runtime': 928.7292, 'train_samples_per_second': 3.23, 'train_steps_per_second': 0.808, 'total_flos': 1567794659328000.0, 'train_loss': 0.7538310953776042, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8639000058174133,
 'eval_accuracy': 0.743,
 'eval_runtime': 77.4951,
 'eval_samples_per_second': 12.904,
 'eval_steps_per_second': 12.904,
 'epoch': 3.0}

## Make a prediction

### Subtask:
Pass the tokenized input through the trained model to get predictions.

## Prepare the input

### Subtask:
Tokenize the user's input using the same tokenizer used for training.

**Reasoning**:
Pass the tokenized input through the trained model to get predictions.

In [ ]:
user_text = "This is a great day!"
tokenized_input = tokenizer(user_text, padding="max_length", truncation=True, return_tensors="pt")

## Interpret the output

### Subtask:
Convert the model's output (logits) into meaningful sentiment labels.

**Reasoning**:
Access the logits from the outputs, find the index of the highest logit, create a mapping from numerical labels to sentiment text, and use the index to get the sentiment text.

In [ ]:
import torch
tokenized_input = {k: v.to(model.device) for k, v in tokenized_input.items()}
outputs = model(**tokenized_input)

In [ ]:
logits = outputs.logits
predicted_class_id = torch.argmax(logits, axis=-1).item()

label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
predicted_sentiment = label_mapping[predicted_class_id]

print(f"The predicted sentiment is: {predicted_sentiment}")

The predicted sentiment is: positive


In [ ]:
import torch

def analyse(user_text):
  tokenized_input = tokenizer(user_text, padding="max_length", truncation=True, return_tensors="pt")
  tokenized_input = {k: v.to(model.device) for k, v in tokenized_input.items()}
  outputs = model(**tokenized_input)
  logits = outputs.logits
  predicted_class_id = torch.argmax(logits, axis=-1).item()

  label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
  predicted_sentiment = label_mapping[predicted_class_id]

  return predicted_sentiment

In [ ]:
answer = analyse("I am very happy")
answer

'positive'